In [ ]:
import jsonlines
import re
import json
import os

In [ ]:
with jsonlines.open('xx/ori/DrugProt/DrugProt_dev.jsonl', 'r') as f:
    gold_datas = [data for data in f]

In [ ]:
with open('xx/ori/DrugProt/schemas.json', 'r') as f:
    schemas = json.load(f)

In [ ]:
gold_tuples = []
for index, data in enumerate(gold_datas):
    for spo in data['spo_list']:
        gold_tuples.append((index, spo['subject'], spo['subject_type'], spo['predicate'], spo['object']['@value'], spo['object_type']['@value']))

In [ ]:
def calc_prf(pred_tuples, gold_tuples):
    pred_tuples = set(pred_tuples)
    gold_tuples = set(gold_tuples)
    corr_tuples = pred_tuples & gold_tuples
    try:
        pre = len(corr_tuples) / len(pred_tuples)
        rec = len(corr_tuples) / len(gold_tuples)
        f1 = 2*pre*rec / (pre+rec)
    except:
        pre = 0
        rec = 0
        f1 = 0
    return pre,rec,f1

In [ ]:
read_files = [
'c_to_hrt_dev_DrugProt.jsonl',
]
read_dir = './pred_result'

In [ ]:
for sub_dir in os.listdir(read_dir):
    for read_file in read_files:
        with jsonlines.open(os.path.join(read_dir, sub_dir, read_file), 'r') as f:
            pred_datas = [data for data in f]
        if 'c_to_hrt' in read_file:
            pattern = r'\(([^|]+?)\|\|([^|]+?)\|\|([^|]+?)\|\|([^|]+?)\|\|([^|]+?)\)\s*\n'
            pred_tuples = []
            for index,data in enumerate(pred_datas):
                finds = re.findall(pattern, data['output'])
                for item in finds:
                    pred_tuples.append((index, item[0], item[1], item[2], item[3], item[4]))

        pre,rec,f1 = calc_prf(pred_tuples, gold_tuples)
        print('pred_file:{}'.format(os.path.join(read_dir, sub_dir, read_file)))
        print('pre:{}\trec:{}\tf1:{}\n'.format(round(pre*100,2), round(rec*100, 2), round(f1*100, 2)))
for i in range(100):
    print()